In [1]:
import shortuuid
import names
import mariadb
import sys
import pandas as pd

In [2]:
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="admin",
        password="admin",
        host='localhost',
        port=3306,
        database="127project"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [3]:
cur.execute("SHOW tables;")
rows = cur.fetchall()
df_tables = pd.DataFrame(data=rows, columns=['TABLES'])
df_tables

,TABLES
0,EXPENSE
1,HAS_GROUP
2,USER
3,USER_FRIEND
4,USER_HAS_GROUP_EXPENSE


# INSERT USER

In [4]:
# INSERT USER
for i in range(25):
    id = shortuuid.uuid()
    name = names.get_full_name()
    cur.execute(
    "INSERT INTO USER (user_id, user_name) VALUES (?, ?);", 
    (id, name))

In [5]:
cur.execute("SELECT * FROM USER;")
rows = cur.fetchall()
df_user = pd.DataFrame(data=rows, columns=['user_id', 'user_name'])
df_user

,user_id,user_name
0,2nA6PhLtqV4dbGyCwUayko,Zandra Seyer
1,2pnrayroy2gRUnJKQBUgp6,Paul Gilliss
2,3Q5UXzj4inwtNuvDfyxC4M,Rosalyn Wiedman
3,8CgGP6ZRhvMydXH3gq32tb,Neva Elumbaugh
4,8EPQ4pCLt2Pa6ZRJxhGdaf,Josue Noble
5,A4yA2os9xRZJT4MT3BVrPc,Catherine Bodin
6,AHEUMBSZrbfGcAPfwCZGqL,Darrell Eversmeyer
7,axg3iEMYRGTT9WqM9Kfn7J,Frank Bishop
8,dNb92xsu8bmepRj3vnf6Fe,Donald Matta
9,E9xQ3VuE9jzNwJ7X7DamRj,Joseph Titus


# INSERT USER_FRIEND 

In [6]:
# getting the ids
cur.execute('SELECT user_id FROM USER;')
rows = cur.fetchall()
df_user_id = pd.DataFrame(data=rows, columns=['user_id'])

In [7]:
import random
list_user_id = df_user_id.user_id
for user_id in list_user_id:
    l = list_user_id.tolist()
    l.remove(user_id)
    p = random.sample(l, 5)
    # print(p)
    for u in p: 
        cur.execute(
        "INSERT INTO USER_FRIEND (user_id, friend) VALUES (?, ?);", 
        (user_id, u))

In [8]:
cur.execute("SELECT * FROM USER_FRIEND;")
rows = cur.fetchall()
df_user = pd.DataFrame(data=rows, columns=['user_id', 'friend'])
df_user

,user_id,friend
0,2nA6PhLtqV4dbGyCwUayko,3Q5UXzj4inwtNuvDfyxC4M
1,2nA6PhLtqV4dbGyCwUayko,FqnCk7MTRLcdwsjNobAWuq
2,2nA6PhLtqV4dbGyCwUayko,m94RPX43vrD9U5NEoFGjVB
3,2nA6PhLtqV4dbGyCwUayko,WQxeVZFBSApMcnoKWLv7HM
4,2nA6PhLtqV4dbGyCwUayko,YWDa6cQs3dte22QJQfoFzv
...,...,...
120,YWDa6cQs3dte22QJQfoFzv,2nA6PhLtqV4dbGyCwUayko
121,YWDa6cQs3dte22QJQfoFzv,8CgGP6ZRhvMydXH3gq32tb
122,YWDa6cQs3dte22QJQfoFzv,8EPQ4pCLt2Pa6ZRJxhGdaf
123,YWDa6cQs3dte22QJQfoFzv,axg3iEMYRGTT9WqM9Kfn7J


# INSERT HAS_GROUP, USER_HAS_GROUP_EXPENSE, EXPENSE 

In [9]:
# getting the ids
cur.execute('SELECT user_id FROM USER;')
rows = cur.fetchall()
df_user_id = pd.DataFrame(data=rows, columns=['user_id']).user_id.tolist()

In [10]:
from faker import Faker
import datetime
fake = Faker()

exp_date = fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d')
exp_date

'2023-03-24'

In [11]:
from mariadb.constants import *

groupno=1
for user_id in df_user_id:
    cur.execute("SELECT friend FROM USER_FRIEND where user_id= ? ;",(user_id,)) # getting the friends of the user
    rows = cur.fetchall()
    df_user_id = pd.DataFrame(data=rows, columns=['user_id']).user_id.tolist() # getting the list

    # random generating of group expsense
    x=random.randrange(1,10)
    for i in range(x):
        group_id = shortuuid.uuid()     
        
        samples  = random.randrange(1,5) # for getting the number of debtor 
        p = random.sample(df_user_id, samples) # getting the random debtors
          
        amount = random.randrange(100,1000)
        group_name =  INDICATOR.NULL if bool(random.getrandbits(1)) else  'Group {}'.format(groupno) # if (randombool) return null else group no.
        groupno+=1
        
        # INSERT HAS_GROUP
        cur.execute("INSERT INTO HAS_GROUP VALUES (? , ? );", (group_id, group_name)) 

        # INSERT USER_HAS_GROUP_EXPENSE
        exp_date = fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d')
        
        expense_id = shortuuid.uuid() # expense id of the creditor
        cur.execute("INSERT INTO USER_HAS_GROUP_EXPENSE VALUES (? , ? , ?);", (user_id, group_id, expense_id)) 
        cur.execute("INSERT INTO EXPENSE VALUES (?, ?, ?, ?, ?)", (expense_id, user_id, amount, 1,exp_date)) # inserting expense

        for debtor in p:
            expense_id = shortuuid.uuid() # expense id of the debtop
            cur.execute("INSERT INTO USER_HAS_GROUP_EXPENSE VALUES (? , ? , ?);", (debtor, group_id, expense_id)) 
            cur.execute("INSERT INTO EXPENSE VALUES (?, ?, ?, ?, ?)", (expense_id, user_id, amount, 0, exp_date)) 

        

In [15]:
conn.commit()

In [13]:
# TRUNCATING THE tables
# cur.execute("TRUNCATE TABLE USER;")
# cur.execute("TRUNCATE TABLE USER_FRIEND;")
# cur.execute("ALTER TABLE TRUNCATE TABLE HAS_GROUP;")


In [14]:
# free resources
# cur.close()
# conn.close()

In [18]:
import os
os.environ['username']

KeyError: 'username'